# Folding Demo

This demo shows how a registered validator can:
 1) Query the top miner uid 
 2) Demonstrate each reward/penalty mechanism/Scoring of the top miner response
 3) Queries the API and/or links to a frontend(if applicable)

In order to do this we perform the following steps:
1. Checks wandb for currently active pdbs (preferably owned by the users’ hotkey)
2. Uses a registered key to initialize a validator neuron
3. Queries the miner hotkeys with the specified pdb
4. Demonstrates how the responses are scores
5. Plots the best configuration


## Requirements
In order to run this notebook you must meet the following requirements:
1. Have a registered key on SN25 (we use opentensor as an example)
2. Have a wandb account 
3. Have a GROMACS 2024 installed


In [ ]:
import os
import wandb
import argparse
import pandas as pd
import bittensor as bt

from inspect import signature

from neurons.validator import Validator
from folding.store import Job
from folding.validators.protein import Protein
from folding.protocol import FoldingSynapse
from folding.validators.reward import get_energies
from folding.utils.ops import get_response_info

WALLET_NAME = 'opentensor'
HOTKEY_NAME = 'main'
SUBTENSOR_NETWORK = 'finney'

parser = argparse.ArgumentParser()
parser.add_argument('--wallet.name', type=str, default=WALLET_NAME)
parser.add_argument('--wallet.hotkey', type=str, default=HOTKEY_NAME)
parser.add_argument('--neuron.axon_off', type=bool, default=True)
config = bt.config(parser=parser)



## Setup the desired wallet

In [ ]:
validator = Validator(config=config)

subtensor = validator.subtensor
metagraph = validator.metagraph
wallet = validator.wallet

wallet

## Extract information from Wandb --> Pandas.DataFrame

In [ ]:
api = wandb.Api()

def load_run(run_path):

    print('Loading run:', run_path)
    run = api.run(run_path)
    df = pd.DataFrame(list(run.scan_history()))
    for col in ['updated_at', 'best_loss_at', 'created_at']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col])
    print(f'+ Loaded {len(df)} records')
    return df


wandb_project = os.path.join(validator.config.wandb.entity, validator.config.wandb.project_name)
netuid = validator.config.netuid
max_runs = 100
min_steps = 10
hotkey = wallet.hotkey.ss58_address
filters = {'state': 'running', 'config.netuid': netuid, "tags": {"$in": [hotkey]}}

print(f'Searching for runs with filters: {filters}')

# Grab runs on folding wandb
for i, run in enumerate(api.runs(wandb_project, filters=filters)):


    if i >= max_runs:
        raise Exception(f'Exceeded max runs {max_runs}')

    num_steps = run.summary.get("_step")
    print(f'run {run}, id: {run.id}, steps: {num_steps}, tags: {run.tags}')
    
    if num_steps is None or num_steps < min_steps:
        continue

    df = load_run('/'.join(run.path))
    version, spec_version, hotkey, netuid_tag, *_ = run.tags
    df['version'] = version
    df['spec_version'] = spec_version
    df['vali_hotkey'] = hotkey
    df['netuid_tag'] = netuid_tag
    df['run_id'] = run.id

df


In [ ]:
# get most recent event log for my validator hotkey
last_event = df.loc[df._step.argmax()]
last_event

In [ ]:
# in particular, we want the pdb_id of the last event and the hotkeys assigned to that job
# NOTE: We cannot guarantee that the top miner is actually assigned to the last job

pdb_id = last_event.pdb_id
hotkeys = last_event.hotkeys

rankings = metagraph.I.argsort(descending=True)

uids = [metagraph.hotkeys.index(hotkey) if hotkey in metagraph.hotkeys else None for hotkey in hotkeys]
incentives = [metagraph.I[uid].item() if uid is not None else None for uid in uids]
rankings = [rankings[uid].item() if uid is not None else None for uid in uids]

df_hotkeys = pd.DataFrame({'hotkey': hotkeys, 'uid': uids, 'incentive': incentives, 'ranking': rankings})

print(f'Miners assigned to protein {pdb_id} job:')
df_hotkeys.sort_values('incentive', ascending=False)


In [ ]:
# Reconstruct the job object from the event log
job = Job(**{k: last_event[k] for k in signature(Job).parameters.keys() if k in last_event})
job


In [ ]:
# Reconstruct the protein object from the job
protein = Protein.from_job(job, config=None)
protein

In [ ]:

# Create a synapse to query the network
synapse = FoldingSynapse(
    pdb_id=protein.pdb_id, md_inputs=protein.md_inputs, mdrun_args=""
)

axons = [metagraph.axons[uid] for uid in uids]

# Make a synchronous to the network with the reconstructed protein
responses = validator.dendrite.query(
    axons=axons,
    synapse=synapse,
    timeout=10,
    deserialize=False,  # decodes the bytestream response inside of md_outputs.
)
responses


In [ ]:

# For now we just want to get the losses, we are not rewarding yet
energies = get_energies(protein=protein, responses=responses, uids=uids)
response_info = get_response_info(responses=responses)